In [1]:
# настраиваем окружение для работы spark
import os

os.environ["SPARK_HOME"] = "/opt/cloudera/parcels/CDH-6.3.1-1.cdh6.3.1.p0.1470567/lib/spark"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
os.environ["HADOOP_CONF_DIR"] = "/etc/hadoop/conf"
#os.environ["PYSPARK_SUBMIT_ARGS"] = "--executor-memory 2G --num-executors 16 --executor-cores 2 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = """--driver-class-path /usr/share/java/postgresql.jar 
--jars /usr/share/java/postgresql.jar --executor-memory 600M --driver-memory 1G --num-executors 1 pyspark-shell"""


In [2]:
# добавляем модуль в путь
import sys
sys.path.append('/opt/cloudera/parcels/CDH-6.3.1-1.cdh6.3.1.p0.1470567/lib/spark/python')

In [3]:
from pyspark.sql import SparkSession

#master = "local"
master = "yarn"
spark = SparkSession.builder.master(master).appName("spark_test").enableHiveSupport().getOrCreate()

In [4]:
df = spark.read.format("jdbc") \
    .option("url","jdbc:postgresql://localhost:7432/scm") \
    .option("dbtable","hosts") \
    .option("user","cloudera-scm") \
    .option("password","2MhalIGcSp") \
    .load()

In [5]:
df.show()

+-------+-----------------------+--------------------+---------+----------+--------+------+-------------------+-----------------+------------------+----------+---------+--------------------+-----------+-----------------+--------------+
|host_id|optimistic_lock_version|     host_identifier|     name|ip_address| rack_id|status|config_container_id|maintenance_count|decommission_count|cluster_id|num_cores|total_phys_mem_bytes|public_name|public_ip_address|cloud_provider|
+-------+-----------------------+--------------------+---------+----------+--------+------+-------------------+-----------------+------------------+----------+---------+--------------------+-----------+-----------------+--------------+
|      1|                     34|a47cfc74-1775-496...|localhost| 127.0.0.1|/default|    NA|                  1|                0|                 0|         1|        2|         10514747392|       null|             null|          null|
+-------+-----------------------+--------------------+--

In [6]:
df.write.format("parquet") \
    .mode("append") \
    .saveAsTable("sp_hosts")

In [7]:
spark.stop()